## MVO Weights
4.57606371261345,9.67698598407907,2.8816471398668,3.76280675093989,6.10005496604961,1.14708159257701,1.63996379334881,3.36411165704113,
2.00796787062891,9.13229658964017,1.37274941345458,2.38654278474067,6.25880838891221,2.14733058728827,9.76228755822564,2.01419193025687,
7.83900398874845,7.72870344184099,2.27786673057532,3.33226384976688,1.03163727917332,7.21709695950646,0.606271511043036,1.73626551968245


In [1]:
import pandas as pd
import numpy as np
import random
import math
from datetime import date,timedelta

In [2]:
def BALANCE(weights):
  #Making sure the total sum of the weights eual to 1
  weights = [w/sum(weights) for w in weights] 
  # Making sure all weights represent proportions that add up to 1
  return weights

In [3]:
def ratio(a,b,c):                       
  #function to calculate ratio i.e. "(returns-(risk_free_rate))/deviation"
  #calculating sharpe ratio
  return (a-c)/b

In [4]:
# def number_of_years(y):#calculates the number of years of the dataset
#   p=y.index[0]         #date of first row in the dataset (datetime format)
#   q=y.index[len(y)-1]  #date of last row in the dataset  (datetime format)
#   return ((q-p).days+1)/365

In [5]:
df=pd.read_csv("2021-01-08.csv",parse_dates=['Date'],index_col='Date')  #Importing Dataset
#df = df.loc["2016-01-01" :"2020-12-31" ]   #Since 2016-01-01, 5y(1234rows till 2020-12-31)
tdf=df.copy()                  #deep copy
df.reset_index(drop=True, inplace=True)
col=list(df.columns)
tdf.shape

(3, 24)

In [6]:
end=tdf.index[2].date()

In [7]:
trading_days=250#len(df)/number_of_years(tdf) 

In [8]:
returnsh=df.pct_change()                  
#Here, returnsh would mean return considered for sharpe ratio
returnsh.fillna(0,inplace=True)

In [9]:
returnsh=returnsh.iloc[1:,:]
returnsh

,ASIANPAINT,BAJFINANCE,BAJAJFINSV,BRITANNIA,DIVISLAB,GRASIM,HCLTECH,HDFCBANK,HINDALCO,HINDUNILVR,...,NESTLEIND,POWERGRID,RELIANCE,TCS,TATACONSUM,TATASTEEL,TECHM,TITAN,ULTRACEMCO,WIPRO
1,-0.004670,0.010079,0.022950,0.003701,-0.019795,0.009395,-0.015999,-0.003027,0.053465,-0.020043,...,-0.020953,0.004588,-0.001619,-0.006128,-0.006777,0.057034,-0.002407,-0.019236,-0.009251,0.000861
2,0.018784,0.000197,0.020427,0.006319,0.014751,0.010515,0.033349,0.010874,-0.017222,0.009435,...,0.009872,0.032987,0.011799,0.029049,0.014480,-0.013351,0.056647,0.004052,0.035903,0.057652


In [10]:
returnso = returnsh.copy()  # this cell considers only NEGATIVE returns so as to calculate sortino ratio
for cols in returnso.columns.tolist():
  for i in range(1, len(df)):
    if returnso[cols][i] > 0:
      returnso[cols][i] = 0


In [11]:
returnso

,ASIANPAINT,BAJFINANCE,BAJAJFINSV,BRITANNIA,DIVISLAB,GRASIM,HCLTECH,HDFCBANK,HINDALCO,HINDUNILVR,...,NESTLEIND,POWERGRID,RELIANCE,TCS,TATACONSUM,TATASTEEL,TECHM,TITAN,ULTRACEMCO,WIPRO
1,-0.00467,0.0,0.0,0.0,-0.019795,0.0,-0.015999,-0.003027,0.000000,-0.020043,...,-0.020953,0.0,-0.001619,-0.006128,-0.006777,0.000000,-0.002407,-0.019236,-0.009251,0.0
2,0.00000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,-0.017222,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,-0.013351,0.000000,0.000000,0.000000,0.0


In [12]:
covmatsh=returnsh.cov()*trading_days     
#Annualised covariance matrix calculated wrt returnsh i.e. used to calculate sharpe ratio
covmatso = returnso.cov() * trading_days

In [13]:
covmatsh

,ASIANPAINT,BAJFINANCE,BAJAJFINSV,BRITANNIA,DIVISLAB,GRASIM,HCLTECH,HDFCBANK,HINDALCO,HINDUNILVR,...,NESTLEIND,POWERGRID,RELIANCE,TCS,TATACONSUM,TATASTEEL,TECHM,TITAN,ULTRACEMCO,WIPRO
ASIANPAINT,0.068759,-0.028972,-0.007398,0.007675,0.101278,0.003284,0.144674,0.040753,-0.207234,0.086420,...,0.090369,0.083255,0.039339,0.103129,0.062319,-0.206349,0.173130,0.068274,0.132376,0.166495
BAJFINANCE,-0.028972,0.012207,0.003117,-0.003234,-0.042674,-0.001384,-0.060959,-0.017172,0.087318,-0.036413,...,-0.038077,-0.035080,-0.016576,-0.043454,-0.026258,0.086945,-0.072949,-0.028767,-0.055777,-0.070153
BAJAJFINSV,-0.007398,0.003117,0.000796,-0.000826,-0.010896,-0.000353,-0.015565,-0.004384,0.022295,-0.009298,...,-0.009722,-0.008957,-0.004232,-0.011095,-0.006705,0.022200,-0.018626,-0.007345,-0.014242,-0.017912
BRITANNIA,0.007675,-0.003234,-0.000826,0.000857,0.011305,0.000367,0.016149,0.004549,-0.023133,0.009647,...,0.010087,0.009293,0.004391,0.011512,0.006956,-0.023034,0.019326,0.007621,0.014776,0.018585
DIVISLAB,0.101278,-0.042674,-0.010896,0.011305,0.149177,0.004837,0.213095,0.060027,-0.305243,0.127291,...,0.133107,0.122630,0.057944,0.151903,0.091793,-0.303939,0.255010,0.100563,0.194981,0.245236
GRASIM,0.003284,-0.001384,-0.000353,0.000367,0.004837,0.000157,0.006909,0.001946,-0.009896,0.004127,...,0.004316,0.003976,0.001879,0.004925,0.002976,-0.009854,0.008268,0.003260,0.006322,0.007951
HCLTECH,0.144674,-0.060959,-0.015565,0.016149,0.213095,0.006909,0.304402,0.085747,-0.436032,0.181833,...,0.190141,0.175174,0.082772,0.216989,0.131124,-0.434169,0.364276,0.143651,0.278526,0.350314
HDFCBANK,0.040753,-0.017172,-0.004384,0.004549,0.060027,0.001946,0.085747,0.024154,-0.122826,0.051221,...,0.053561,0.049345,0.023316,0.061124,0.036936,-0.122302,0.102613,0.040465,0.078458,0.098680
HINDALCO,-0.207234,0.087318,0.022295,-0.023133,-0.305243,-0.009896,-0.436032,-0.122826,0.624581,-0.260461,...,-0.272362,-0.250922,-0.118564,-0.310821,-0.187824,0.621913,-0.521797,-0.205770,-0.398967,-0.501798
HINDUNILVR,0.086420,-0.036413,-0.009298,0.009647,0.127291,0.004127,0.181833,0.051221,-0.260461,0.108617,...,0.113579,0.104639,0.049443,0.129617,0.078326,-0.259348,0.217598,0.085809,0.166376,0.209258


In [14]:
risk_free_rate = 0.0358 #initializing risk free rate that will be used in calculating both the ratios (absolute value)
#referred from url: https://www.rbi.org.in/Scripts/BS_NSDPDisplay.aspx?param=4&Id=24292
#In the above url, the 364 (1 year) day treasury bill is 3.58% , when taken absolute value => 0.0358
# (improved)

In [15]:
df
stocks=df.shape[1]
stocks

24

# Sharpe


In [16]:
global_warr_sortino=[]
global_war_sharpe=[]
sharpe_portfolio_return=[]
sharpe_portfolio_risk=[]
sharpe_portfolio_shratio=[]
sharpe_portfolio_stockWeights=[]

In [17]:

fitness=0
#for each iteration

weights=np.array(BALANCE(np.array([4.57606371261345,9.67698598407907,2.8816471398668,3.76280675093989,6.10005496604961,1.14708159257701,1.63996379334881,3.36411165704113,2.00796787062891,9.13229658964017,1.37274941345458,2.38654278474067,6.25880838891221,2.14733058728827,9.76228755822564,2.01419193025687,7.83900398874845,7.72870344184099,2.27786673057532,3.33226384976688,1.03163727917332,7.21709695950646,0.606271511043036,1.73626551968245
])))

#calculating annulaised portfolio return
returns_temp = np.sum(returnsh.mean()*weights)*trading_days 

#calculating portfolio varience wrt calculating sharpe ratio
varsh=np.dot(weights.T,np.dot(covmatsh,weights))   

#portfolio risk
volatility_temp = np.sqrt(varsh)      

#Calculating fitness value(ie sharpe ratio)
fitness = ratio(returns_temp,volatility_temp,risk_free_rate)

#Initializing the intial fitness value as the best fitness value(sharpe_pbest)


sharpe_portfolio_return.append(returns_temp)
sharpe_portfolio_risk.append(volatility_temp)
sharpe_portfolio_shratio.append(fitness)
sharpe_portfolio_stockWeights.append(weights)



In [18]:
sharpe_portfolio = {'Returns' : sharpe_portfolio_return, 'Standard Deviation' : sharpe_portfolio_risk,  'Sharpe Ratio' : sharpe_portfolio_shratio}  

for counter,symbol in enumerate(df.columns):
  sharpe_portfolio[symbol + " Weight"] = [Weight[counter] for Weight in sharpe_portfolio_stockWeights]
sharpe_pc = pd.DataFrame(sharpe_portfolio)
sharpe_optimal=sharpe_pc.iloc[sharpe_pc['Sharpe Ratio'].idxmax()]
sharpe_optimal=pd.DataFrame(sharpe_optimal)
sharpe_optimal.to_csv("sharpe_optimal.csv")

sharpe_pc

,Returns,Standard Deviation,Sharpe Ratio,ASIANPAINT Weight,BAJFINANCE Weight,BAJAJFINSV Weight,BRITANNIA Weight,DIVISLAB Weight,GRASIM Weight,HCLTECH Weight,...,NESTLEIND Weight,POWERGRID Weight,RELIANCE Weight,TCS Weight,TATACONSUM Weight,TATASTEEL Weight,TECHM Weight,TITAN Weight,ULTRACEMCO Weight,WIPRO Weight
0,1.201507,0.162203,7.186704,0.045761,0.09677,0.028816,0.037628,0.061001,0.011471,0.0164,...,0.097623,0.020142,0.07839,0.077287,0.022779,0.033323,0.010316,0.072171,0.006063,0.017363


In [19]:
sharpe_pc.to_csv(str(end)+'MVOsharpe_A.csv')

# Sortino

In [20]:
global_warr_sortino=[]
sortino_portfolio_return=[]
sortino_portfolio_risk=[]
sortino_portfolio_soratio=[]
sortino_portfolio_stockWeights=[]

In [21]:

fitness=0
#for each iteration


#Making sure that the total amount of pheromone equals 1 
weights=np.array(BALANCE([4.57606371261345,9.67698598407907,2.8816471398668,3.76280675093989,6.10005496604961,1.14708159257701,1.63996379334881,3.36411165704113,2.00796787062891,9.13229658964017,1.37274941345458,2.38654278474067,6.25880838891221,2.14733058728827,9.76228755822564,2.01419193025687,7.83900398874845,7.72870344184099,2.27786673057532,3.33226384976688,1.03163727917332,7.21709695950646,0.606271511043036,1.73626551968245
]))

#calculating annulaised portfolio return
returns_temp = np.sum(returnsh.mean()*weights)*trading_days 

#calculating portfolio varience wrt calculating sharpe ratio
varso=np.dot(weights.T,np.dot(covmatso,weights))   

#portfolio risk
volatility_temp = np.sqrt(varso)      

#Calculating fitness value(ie sortino ratio)
fitness = ratio(returns_temp,volatility_temp,risk_free_rate)

#Initializing the intial fitness value as the best fitness value(sortino_pbest)

sortino_portfolio_return.append(returns_temp)
sortino_portfolio_risk.append(volatility_temp)
sortino_portfolio_soratio.append(fitness)
sortino_portfolio_stockWeights.append(weights)


In [22]:
sortino_portfolio = {'Returns' : sortino_portfolio_return, 'Semi Deviation' : sortino_portfolio_risk,  'Sortino Ratio' : sortino_portfolio_soratio}  

for counter,symbol in enumerate(df.columns):
  sortino_portfolio[symbol + " Weight"] = [Weight[counter] for Weight in sortino_portfolio_stockWeights]
sortino_pc = pd.DataFrame(sortino_portfolio)
sortino_optimal=sortino_pc.iloc[sortino_pc['Sortino Ratio'].idxmax()]
sortino_optimal=pd.DataFrame(sortino_optimal)
sortino_optimal.to_csv("sortino_optimal.csv")
sortino_pc


,Returns,Semi Deviation,Sortino Ratio,ASIANPAINT Weight,BAJFINANCE Weight,BAJAJFINSV Weight,BRITANNIA Weight,DIVISLAB Weight,GRASIM Weight,HCLTECH Weight,...,NESTLEIND Weight,POWERGRID Weight,RELIANCE Weight,TCS Weight,TATACONSUM Weight,TATASTEEL Weight,TECHM Weight,TITAN Weight,ULTRACEMCO Weight,WIPRO Weight
0,1.201507,0.080643,14.455161,0.045761,0.09677,0.028816,0.037628,0.061001,0.011471,0.0164,...,0.097623,0.020142,0.07839,0.077287,0.022779,0.033323,0.010316,0.072171,0.006063,0.017363


In [23]:

sortino_pc.to_csv(str(end)+'MVOsortino_A.csv')